In [1]:
import random
import time
import os

import cv2
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from IPython import display

from torchvision.transforms import transforms
import torch.utils.data
from torchsummary import summary

import matplotlib.pyplot as plt
import itertools

from pytorchCycleGANandpix2pix.models import networks as offnets
from pytorchCycleGANandpix2pix.util.image_pool import ImagePool
from pytorchCycleGANandpix2pix.util.util import tensor2im

import cyclegan_models

In [14]:
def revert_normalisation(tensor):
    return (tensor.permute(1, 2, 0) + 1) / 2

def revert_normalisation_batch(tensors):
    return (tensors.permute(0, 2, 3, 1) + 1) / 2

In [3]:
device = "cuda"

In [4]:
model_dir = "./runs/1677749332.549509/115"
x_to_y = True

In [5]:
model = cyclegan_models.Generator()
model.load_state_dict(torch.load(f"{model_dir}/{'G' if x_to_y else 'F'}.pth"))
model.eval()
model.to(device)
;

''

In [6]:
transfer_input = [
    "./cyclegan_original_data/horse2zebra/testA/n02381460_840.jpg",
    "./cyclegan_original_data/horse2zebra/testA/n02381460_2100.jpg",
]

In [80]:
eval_transform = transforms.Compose([
    transforms.Resize(128)
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def transfer_to_batch(images):
    transformed = torch.stack([eval_transform(image) for image in images]).to(device)
    
    with torch.no_grad():
        transferred = model(transformed).detach().cpu()
    
    return revert_normalisation_batch(transferred)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1488528856.py, line 2)

In [89]:
def transfer_to_video(video_loc):
    inp_vid = cv2.VideoCapture(video_loc)
    inp_fps = int(inp_vid.get(cv2.CAP_PROP_FPS))
    inp_len = int(inp_vid.get(cv2.CAP_PROP_FRAME_COUNT))
    
    success, frame = inp_vid.read()
    
    height, width, channels = frame.shape
    
    split_loc = video_loc.split(".")
    ext = split_loc[-1]
    no_ext_loc = ".".join(split_loc[:-1])
    save_loc = f"{no_ext_loc}_{time.time():.0f}.{ext}"
    out_vid = cv2.VideoWriter(save_loc, -1, inp_fps, (width, height))
    
    print(save_loc)
    
    batched = []
    c = 0
    
    while success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        batched.append(frame.copy())
        
        if len(batched) == 4:
            stacked = np.stack(batched)
            transferred = transfer_to_batch(stacked)
            
            for new_frame in transferred:
                new_frame = (new_frame.numpy() * 255).astype(np.uint8)
                new_frame = cv2.cvtColor(new_frame, cv2.COLOR_RGB2BGR)
                
                cv2.imwrite(f"./examples/{time.time()}.png", new_frame)
                out_vid.write(new_frame)
                
            batched = []
        
        success, frame = inp_vid.read()
        c += 1
        
        if c == 40:
            break
    
    out_vid.release()
    inp_vid.release()

In [90]:
transfer_to_video("./examples/horse_vid.mp4")

./examples/horse_vid_1677771975.mp4


KeyboardInterrupt: 